In [1]:
# Necessary imports
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Layout, widgets, Dropdown, VBox, HBox, FloatText, interactive
import scipy.signal
import time
import scipy.special

# Root Raised Cosine filter
def rootRaisedCosine(nsamp, roll_off, delay):
    F0 = 0.5 / nsamp
    Br = 1
    Fs = Br * nsamp
    Td = 1 / Br
    Ts = 1 / Fs
    F1 = F0 * (1 - roll_off)
    F2 = F0 * (1 + roll_off)
    filter_order = 2 * nsamp * delay

    t = np.arange(0, filter_order, Td)
    h = []
    for i in range(len(t)):
        t_shifted = t[i] - filter_order / 2
        if t_shifted == 0:
            h.append(np.sqrt(2 * F0) *(1 + roll_off * ((4 / np.pi) - 1)))
        elif t_shifted == 1 / 8 / roll_off / F0 or t_shifted == - 1 / 8 / roll_off / F0 :
            h.append((roll_off * np.sqrt(F0)) * ((1 + 2 / np.pi) * np.sin(np.pi / 4 / roll_off) + (1 - 2 / np.pi) * np.cos(np.pi / 4 / roll_off)))
        else:
            factor1 = np.sqrt(2 * F0) / (1 - 64 * roll_off * roll_off * F0 * F0 * t_shifted * t_shifted)
            factor2 = np.sin(2 * np.pi * F1 * t_shifted) / (2 * np.pi * F0 * t_shifted)
            factor3 = (4 * roll_off / np.pi) * np.cos(2 * np.pi * F2 * t_shifted)
            h.append(factor1 * (factor2 + factor3))
    return h

# Bit Error Rate Calculation for QAM
def ber_qam(EbNo, M, roll_off, F1, F2, Br):
    F1 = F1 * 1e6  # Convert MHz to Hz
    F2 = F2 * 1e6  # Convert MHz to Hz
    Br = Br * 1e6  # Convert Mbps to Hz
    W = F2 - F1  # Bandwidth in Hz
    fc = F1 + W / 2  # Carrier frequency
    nsamp = int(np.ceil(2 * F2 / Br)) + 7  # Number of samples per symbol

    L = int(np.sqrt(M))
    l = np.log2(L)
    k = 2 * l
    Nsymb = 10000
    SNR = EbNo - 10 * np.log10(nsamp / k / 2)
    core = [1+1j, 1-1j, -1+1j, -1-1j]
    mapping = core[:]
    if l > 1:
        for j in range(1, int(l)):
            mapping = list(map(lambda x: x + j * 2 * core[0], mapping))
            conj_arr = np.conj(mapping)
            mapping = mapping + conj_arr.tolist()
            conj_arr = -np.conj(mapping)
            mapping = mapping + conj_arr.tolist()

    # Generate random sequence
    x = np.floor(2 * np.random.rand(int(k * Nsymb), 1))
    x_temp = np.reshape(x, (int(len(x) / (k)), int(k)))
    xsym = []

    for i in range(len(x_temp)):
        my_str = ''
        y = x_temp[i]
        for j in range(int(np.log2(M))):
            my_str = my_str + str(int(y[j]))
        a = int(my_str, 2)
        xsym = xsym + [a]

    y = []
    for n in range(len(xsym)):
        y = y + [mapping[xsym[n]]]

    delay = 10
    filtorder = delay * nsamp * 2

    shaping_filter = rootRaisedCosine(nsamp, roll_off, delay)
    ytx = scipy.signal.upfirdn([1], y, nsamp)  # upsample
    ytx = np.convolve(ytx, shaping_filter)
    m = np.arange(1, len(ytx) + 1)
    s = np.real(np.multiply(ytx, np.exp(1j * 2 * np.pi * fc * m / nsamp)))

    s_matrix = np.matrix(s)
    s_matrix = s_matrix.getH()
    s_list = s_matrix.tolist()
    Ps = 10 * np.log10(np.matmul(s, s_list) / len(s))
    Pn = Ps - SNR

    n = np.sqrt(10**(Pn / 10)) * np.random.randn(1, len(ytx))
    snoisy = s + n

    # Receiver
    yrx = 2 * np.multiply(snoisy, np.exp(-1j * 2 * np.pi * fc * m / nsamp))
    yrx = yrx[0, :]
    yrx = np.convolve(yrx, shaping_filter)
    yrx = yrx[::nsamp]

    yrx = yrx[2 * delay + 0:len(yrx) - 2 * delay]

    yi = yrx.copy()
    yq = np.imag(yi)
    yi = np.real(yi)

    xrx = []
    q = np.arange(-L + 1, L, 2)

    for n in range(len(yrx)):
        differences = np.abs(q - yi[n])
        m = min(differences)
        [index], = np.where(differences == m)
        yi[n] = q[index]
        differences = np.abs(q - yq[n])
        m = min(differences)
        [index], = np.where(differences == m)
        yq[n] = q[index]
    error = 0
    for i in range(len(yrx)):
        if y[i] != yi[i] + yq[i] * 1j:
            error += 1 
    return error / len(x)

# Plot BER Curve for QAM
def plot_ber_qam(M, roll_off, F1, F2, Br):
    if F1 >= F2:
        print("Warning: F1 should be less than F2.")
        return

    ber_exp = []
    ber_th = []
    L = int(np.sqrt(M))
    for i in range(1, 15):
        ber_exp.append(ber_qam(i, M, roll_off, F1, F2, Br))
        ber_th.append(((L - 1) /(L*np.log2(L)) * scipy.special.erfc(np.sqrt(3 * np.log2(L) / (L * L - 1) * 10**(i/10)))))

    plt.figure(figsize=(10, 8))
    plt.semilogy(range(1, 15), ber_th)  # Plot theoretical BER as a line
    plt.semilogy(range(1, 15), ber_exp, 'o')  # Plot experimental BER as points
    plt.legend(['Theoretical', 'Simulation'])
    plt.xlabel('Eb/N0(db)')
    plt.ylabel('Bit Error Probability')
    plt.title(f'BER curve for {M}-QAM')
    plt.grid(which='both')
    plt.show()

# Define QAM options
qam_options = {'4-QAM': 4, '16-QAM': 16, '64-QAM': 64}
qam_selector = widgets.Dropdown(options=qam_options, value=16, description='QAM Type:')

# Define additional input boxes for roll-off, F1, F2, and Br
roll_off_input = widgets.FloatText(value=0.25, description='Roll-off:')
F1_input = widgets.FloatText(value=6.75, description='F1: (MHz)')
F2_input = widgets.FloatText(value=9.25, description='F2: (MHz)')
Br_input = widgets.FloatText(value=10, description='Br: (Mbps)')

# Create interactive widget
interactive_plot = interactive(plot_ber_qam, M=qam_selector, roll_off=roll_off_input, F1=F1_input, F2=F2_input, Br=Br_input)

input_widgets = VBox([qam_selector, roll_off_input, F1_input, F2_input, Br_input], layout=Layout(width='auto'))
plot_output = interactive_plot.children[-1]

# Display the UI components
display(input_widgets, plot_output)


Output()